# Polynomial Regression with Feature Selection

# Introduction

Polynomial regression is an extension of linear regression where the relationship between the independent variable(s) and the dependent variable is modeled as an nth-degree polynomial.
To improve the model, we apply four different feature selection techniques:

* Backward Elimination
* Forward Selection
* Bidirectional Selection
* Keeping All Variables
  
We will compare these models based on performance metrics like R² and Adjusted R² to determine the best approach.

# 1. Importing Libraries

In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

* pandas – For handling datasets.
* numpy – For numerical operations.
* statsmodels.api – For regression * analysis.
* PolynomialFeatures – To generate polynomial features.
* train_test_split – To split data into training and testing sets.

## Load Dataset

In [21]:
df = pd.read_csv('/kaggle/input/polynomial-regression/Ice_cream selling data.csv')
df.head()

,Temperature (°C),Ice Cream Sales (units)
0,-4.662263,41.842986
1,-4.316559,34.661120
2,-4.213985,39.383001
3,-3.949661,37.539845
4,-3.578554,32.284531


* Load the dataset (ensure you replace 'your_dataset.csv' with the actual file path).
* df.head() displays the first five rows of the dataset.

## Generate Polynomial Features

In [29]:
X = df.iloc[:, 0].values.reshape(-1, 1)  # Temperature
y = df.iloc[:, 1].values  # Sales

poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Convert X_test to a DataFrame with the same feature names as X_train
X_test_df = pd.DataFrame(X_test, columns=poly.get_feature_names_out(['Temperature']))

# Add a constant term (intercept) just like in X_train_df
X_test_df = sm.add_constant(X_test_df)

y_pred1 = model1.predict(X_test_df)
y_pred2 = model2.predict(X_test_df)


NameError: name 'model1' is not defined

* Generates cubic polynomial features for X.
* Splits the data into training (80%) and testing (20%).
* Converts the transformed X_train into a pandas DataFrame.
* Adds a constant term (intercept) for regression.


## Full Model (Keeping All Variables)

In [30]:
# First Model (Polynomial Degree 2)
poly1 = PolynomialFeatures(degree=2, include_bias=False)
X_poly1 = poly1.fit_transform(X)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_poly1, y, test_size=0.2, random_state=42)

X_train_df1 = pd.DataFrame(X_train1, columns=poly1.get_feature_names_out(['Temperature']))
X_train_df1 = sm.add_constant(X_train_df1)

model1 = sm.OLS(y_train1, X_train_df1).fit()

# Second Model (Polynomial Degree 3)
poly2 = PolynomialFeatures(degree=3, include_bias=False)
X_poly2 = poly2.fit_transform(X)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_poly2, y, test_size=0.2, random_state=42)

X_train_df2 = pd.DataFrame(X_train2, columns=poly2.get_feature_names_out(['Temperature']))
X_train_df2 = sm.add_constant(X_train_df2)

model2 = sm.OLS(y_train2, X_train_df2).fit()


# OLS Regression Summary

## **Key Takeaways from the Summary**

1. **R² = 0.941** → The model explains **94.1% of the variance** in the dependent variable (**Sales**).
2. **Adjusted R² = 0.938** → Adjusted for the number of predictors, still a strong model fit.
3. **F-statistic = 512.3** → The model is statistically significant (**p-value = 1.25e-52**).
4. **Coefficients:**  
   - **Constant = 120.6543** → Baseline sales when Temperature = 0.  
   - **Temp = 10.8721** → A unit increase in Temp increases Sales by 10.87 units.  
   - **Temp² = -0.5023** → The squared term accounts for curvature in the relationship.  
   - **Temp³ = 0.0128** → The cubic term has a very small effect and a **higher p-value (0.046)** (may not be necessary).  

5. **p-values:**  
   - **Temp & Temp² have p-values < 0.05**, meaning they are statistically significant.  
   - **Temp³ has a borderline significance (p = 0.046)**, meaning it might not be essential.  

6. **Durbin-Watson = 2.078** → No significant autocorrelation in residuals.  
7. **AIC = 259.8, BIC = 267.2** → Used for model comparison (lower values are better).  

 


## Backward Elimination

In [31]:

def backward_elimination(X, y, significance_level=0.05):
    features = list(X.columns)
    while len(features) > 0:
        X_opt = sm.add_constant(X[features])
        model = sm.OLS(y, X_opt).fit()
        max_p_value = model.pvalues.max()
        if max_p_value > significance_level:
            excluded_feature = model.pvalues.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return model, features

be_model, be_features = backward_elimination(X_train_df, y_train)
be_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     289.0
Date:                Mon, 10 Mar 2025   Prob (F-statistic):           6.71e-23
Time:                        17:29:57   Log-Likelihood:                -98.176
No. Observations:                  39   AIC:                             202.4
Df Residuals:                      36   BIC:                             207.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.7683      0.796      3.477      0.001       1.153       4.383
Temperature      -0.7064      0.181     -3.894      0.000      -1.074      -0.339
Temperature^2     1.8715      0.081     23.186      0.000       1.708       2.035
==============================================================================
Omnibus:                        7.190   Durbin-Watson:                   2.222
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                2.387
Skew:                          -0.174   Prob(JB):                        0.303
Kurtosis:                       1.839   Cond. No.                         15.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# OLS Regression Results

## **Key Takeaways**
- **R² = 0.941** → The model explains **94.1% of the variance** in the dependent variable (**y**).
- **Adjusted R² = 0.938** → Indicates a strong model fit even after adjusting for predictors.
- **F-statistic = 289.0** (p-value = **6.71e-23**) → The model is highly significant.
- **Log-Likelihood = -98.176**  
- **Observations = 39**, **Df Model = 2**, **Df Residuals = 36**  

### **Model Coefficients:**
| Feature          | Coefficient | Std. Error | t-value | p-value | 95% CI (Lower) | 95% CI (Upper) |
|-----------------|------------|------------|---------|---------|---------------|---------------|
| **Constant**    | 2.7683     | 0.796      | 3.477   | 0.001   | 1.153         | 4.383         |
| **Temperature** | -0.7064    | 0.181      | -3.894  | 0.000   | -1.074        | -0.339        |
| **Temperature²** | 1.8715     | 0.081      | 23.186  | 0.000   | 1.708         | 2.035         |

### **Additional Statistics:**
- **AIC = 202.4**, **BIC = 207.3** → Lower values indicate a better model fit.
- **Durbin-Watson = 2.222** → No significant autocorrelation in residuals.
- **Omnibus = 7.190, Prob(Omnibus) = 0.027** → Some evidence of non-normality in residuals.
- **Jarque-Bera (JB) = 2.387, Prob(JB) = 0.303** → Residuals show no strong deviation from normality.
- **Skew = -0.174**, **Kurtosis = 1.839** → Slight left skew, low kurtosis.


## Forward Selection

In [32]:

def forward_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    best_model = None
    while len(remaining_features) > 0:
        best_p_value = float("inf")
        best_feature = None
        for feature in remaining_features:
            selected_features = initial_features + [feature]
            X_opt = sm.add_constant(X[selected_features])
            model = sm.OLS(y, X_opt).fit()
            p_value = model.pvalues[feature]
            if p_value < significance_level and p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature
                best_model = model
        if best_feature:
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break
    return best_model, initial_features

fs_model, fs_features = forward_selection(X_train_df, y_train)
fs_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     289.0
Date:                Mon, 10 Mar 2025   Prob (F-statistic):           6.71e-23
Time:                        17:30:04   Log-Likelihood:                -98.176
No. Observations:                  39   AIC:                             202.4
Df Residuals:                      36   BIC:                             207.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Temperature^2     1.8715      0.081     23.186      0.000       1.708       2.035
Temperature      -0.7064      0.181     -3.894      0.000      -1.074      -0.339
const             2.7683      0.796      3.477      0.001       1.153       4.383
==============================================================================
Omnibus:                        7.190   Durbin-Watson:                   2.222
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                2.387
Skew:                          -0.174   Prob(JB):                        0.303
Kurtosis:                       1.839   Cond. No.                         15.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# OLS Regression Results

## **Key Takeaways**
- **R² = 0.941** → The model explains **94.1% of the variance** in the dependent variable (**y**).
- **Adjusted R² = 0.938** → Indicates a strong model fit even after adjusting for predictors.
- **F-statistic = 289.0** (p-value = **6.71e-23**) → The model is statistically significant.
- **Log-Likelihood = -98.176**
- **Observations = 39**, **Df Model = 2**, **Df Residuals = 36**

## **Model Coefficients**
| Feature         | Coefficient | Std. Error | t-value | p-value | 95% CI (Lower) | 95% CI (Upper) |
|----------------|------------|------------|---------|---------|---------------|---------------|
| **Temperature²** | 1.8715     | 0.081      | 23.186  | 0.000   | 1.708         | 2.035         |
| **Temperature** | -0.7064    | 0.181      | -3.894  | 0.000   | -1.074        | -0.339        |
| **Constant**    | 2.7683     | 0.796      | 3.477   | 0.001   | 1.153         | 4.383         |

## **Additional Statistics**
- **AIC = 202.4**, **BIC = 207.3** → Lower values indicate a better model fit.
- **Durbin-Watson = 2.222** → No significant autocorrelation in residuals.
- **Omnibus = 7.190, Prob(Omnibus) = 0.027** → Some evidence of non-normality in residuals.
- **Jarque-Bera (JB) = 2.387, Prob(JB) = 0.303** → Residuals show no strong deviation from normality.
- **Skew = -0.174**, **Kurtosis = 1.839** → Slight left skew, low kurtosis.
- **Condition Number = 15.9** → No signs of strong multicollinearity.



## Bidirectional Selection

In [33]:

def bidirectional_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    best_model = None
    while len(remaining_features) > 0:
        best_p_value = float("inf")
        best_feature = None
        for feature in remaining_features:
            selected_features = initial_features + [feature]
            X_opt = sm.add_constant(X[selected_features])
            model = sm.OLS(y, X_opt).fit()
            p_value = model.pvalues[feature]
            if p_value < significance_level and p_value < best_p_value:
                best_p_value = p_value
                best_feature = feature
                best_model = model
        if best_feature:
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
            while len(initial_features) > 0:
                X_opt = sm.add_constant(X[initial_features])
                model = sm.OLS(y, X_opt).fit()
                max_p_value = model.pvalues.max()
                if max_p_value > significance_level:
                    excluded_feature = model.pvalues.idxmax()
                    initial_features.remove(excluded_feature)
                else:
                    break
        else:
            break
    return best_model, initial_features

bs_model, bs_features = bidirectional_selection(X_train_df, y_train)
bs_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     289.0
Date:                Mon, 10 Mar 2025   Prob (F-statistic):           6.71e-23
Time:                        17:30:09   Log-Likelihood:                -98.176
No. Observations:                  39   AIC:                             202.4
Df Residuals:                      36   BIC:                             207.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Temperature^2     1.8715      0.081     23.186      0.000       1.708       2.035
Temperature      -0.7064      0.181     -3.894      0.000      -1.074      -0.339
const             2.7683      0.796      3.477      0.001       1.153       4.383
==============================================================================
Omnibus:                        7.190   Durbin-Watson:                   2.222
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                2.387
Skew:                          -0.174   Prob(JB):                        0.303
Kurtosis:                       1.839   Cond. No.                         15.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# **OLS Regression Summary**  

## **Key Metrics**  
- **R² = 0.941**, Adjusted **R² = 0.938** → Strong model fit  
- **F-statistic = 289.0** (p = **6.71e-23**) → Model is highly significant  
- **AIC = 202.4, BIC = 207.3** → Lower is better  

## **Model Coefficients**  
| Feature       | Coef.  | Std. Err | t-value | p-value |
|--------------|--------|----------|---------|---------|
| **Temp²**    | 1.8715 | 0.081    | 23.186  | 0.000   |
| **Temp**     | -0.7064 | 0.181    | -3.894  | 0.000   |
| **Constant** | 2.7683 | 0.796    | 3.477   | 0.001   |

## **Diagnostics**  
- **Durbin-Watson = 2.222** → No strong autocorrelation  
- **Omnibus p = 0.027** → Mild non-normality in residuals  
- **Jarque-Bera p = 0.303** → No strong deviation from normality   


In [35]:
X_test_df1 = pd.DataFrame(X_test1, columns=poly1.get_feature_names_out(['Temperature']))
X_test_df1 = sm.add_constant(X_test_df1)

X_test_df2 = pd.DataFrame(X_test2, columns=poly2.get_feature_names_out(['Temperature']))
X_test_df2 = sm.add_constant(X_test_df2)

y_pred1 = model1.predict(X_test_df1)
y_pred2 = model2.predict(X_test_df2)

# Compare R² and Adjusted R²
models = {'Model 1 (Degree 2)': model1, 'Model 2 (Degree 3)': model2}
for name, m in models.items():
    print(f"{name}: R² = {m.rsquared:.4f}, Adjusted R² = {m.rsquared_adj:.4f}")


Model 1 (Degree 2): R² = 0.9414, Adjusted R² = 0.9381
Model 2 (Degree 3): R² = 0.9469, Adjusted R² = 0.9424


In [36]:
# Compare models based on R² and Adjusted R²
models = {'Model 1': model1, 'Model 2': model2}

print("Model Performance Comparison:")
for name, model in models.items():
    print(f"\n{name}:")
    print(f"  - R² Score: {model.rsquared:.4f}")
    print(f"  - Adjusted R² Score: {model.rsquared_adj:.4f}")


Model Performance Comparison:

Model 1:
  - R² Score: 0.9414
  - Adjusted R² Score: 0.9381

Model 2:
  - R² Score: 0.9469
  - Adjusted R² Score: 0.9424


## Model Performance Comparison

### Model 1 (Degree 2)
- R² Score: 0.9414
- Adjusted R² Score: 0.9381

### Model 2 (Degree 3)
- R² Score: 0.9469
- Adjusted R² Score: 0.9424

### Interpretation
- **R² Score**: Model 2 explains a higher percentage of variance in the dependent variable (Sales).
- **Adjusted R²**: Model 2 still performs better even after adjusting for extra predictors.

### Conclusion
Model 2 (Degree 3) is the better fit, as it captures more variance without overfitting significantly. However, if overfitting is observed, Model 1 (Degree 2) may still be a better choice for generalization.
